### model training and testing

In [1]:
import os
import warnings
from math import sqrt

import numpy as np
import pandas as pd
import pins
import pyodbc
from dotenv import load_dotenv
from pprint import pprint
from rsconnect.api import RSConnectServer
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.model_selection import TimeSeriesSplit, train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from vetiver import VetiverModel, deploy_rsconnect, pin_read_write

In [2]:
# import sys

# sys.path.append(".")

# from utils import DataCleaner

In [3]:
warnings.filterwarnings('ignore')
load_dotenv(override=True)

rsc_server = os.getenv("CONNECT_SERVER")
rsc_key = os.getenv("CONNECT_API_KEY")
connect_server = RSConnectServer(url=rsc_server,api_key=rsc_key)

## Raw data

Read in the raw data from the database.

In [4]:
# read table built by R ETL process in the database
connection = pyodbc.connect('DSN=Content DB')

sql = "select * from bike_model_data where date in (select distinct date from bike_model_data order by date desc limit 12);"

all_days = pd.read_sql_query(sql, connection)
all_days.sort_values(by='date', inplace=True, ascending=False)
all_days = all_days.reset_index(drop=True)

all_days

,id,hour,date,month,dow,n_bikes,lat,lon
0,453,4.0,2023-06-09,6.0,Friday,16.0,38.919086,-77.034502
1,299,0.0,2023-06-09,6.0,Friday,1.0,39.110314,-77.182669
2,298,8.0,2023-06-09,6.0,Friday,13.0,39.114688,-77.171487
3,298,10.0,2023-06-09,6.0,Friday,13.0,39.114688,-77.171487
4,298,12.0,2023-06-09,6.0,Friday,13.0,39.114688,-77.171487
...,...,...,...,...,...,...,...,...
39093,125,8.0,2023-05-29,5.0,Monday,6.0,38.897857,-77.026975
39094,326,16.0,2023-05-29,5.0,Monday,8.0,38.964992,-77.103381
39095,326,18.0,2023-05-29,5.0,Monday,8.0,38.964992,-77.103381
39096,326,20.0,2023-05-29,5.0,Monday,8.0,38.964992,-77.103381


In [5]:
# Only keep required columns
# all_days = all_days[[
#     'lat',
#     'lon',
#     'date',
#     'dow',
#     'hour',
#     'month',
#     'n_bikes'
# ]]

all_days

,id,hour,date,month,dow,n_bikes,lat,lon
0,453,4.0,2023-06-09,6.0,Friday,16.0,38.919086,-77.034502
1,299,0.0,2023-06-09,6.0,Friday,1.0,39.110314,-77.182669
2,298,8.0,2023-06-09,6.0,Friday,13.0,39.114688,-77.171487
3,298,10.0,2023-06-09,6.0,Friday,13.0,39.114688,-77.171487
4,298,12.0,2023-06-09,6.0,Friday,13.0,39.114688,-77.171487
...,...,...,...,...,...,...,...,...
39093,125,8.0,2023-05-29,5.0,Monday,6.0,38.897857,-77.026975
39094,326,16.0,2023-05-29,5.0,Monday,8.0,38.964992,-77.103381
39095,326,18.0,2023-05-29,5.0,Monday,8.0,38.964992,-77.103381
39096,326,20.0,2023-05-29,5.0,Monday,8.0,38.964992,-77.103381


## Train test split

In [6]:
X = all_days.drop(columns=["n_bikes"])
y = all_days[["n_bikes"]]

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [7]:
X_train

,id,hour,date,month,dow,lat,lon
6699,32,16.0,2023-06-07,6.0,Wednesday,38.889600,-76.976900
29307,142,18.0,2023-05-31,5.0,Wednesday,38.886372,-77.017701
37713,396,12.0,2023-05-29,5.0,Monday,38.799267,-77.044700
10688,83,0.0,2023-06-06,6.0,Tuesday,38.899032,-77.033354
13367,183,12.0,2023-06-05,6.0,Monday,38.905090,-76.994100
...,...,...,...,...,...,...,...
15795,59,2.0,2023-06-04,6.0,Sunday,38.912100,-77.038700
34569,382,12.0,2023-05-30,5.0,Tuesday,38.813485,-77.049468
19323,409,22.0,2023-06-03,6.0,Saturday,38.890544,-77.049379
36046,261,14.0,2023-05-29,5.0,Monday,39.096312,-77.192672


In [8]:
y_train

,n_bikes
6699,10.0
29307,37.0
37713,14.0
10688,1.0
13367,8.0
...,...
15795,8.0
34569,7.0
19323,12.0
36046,8.0


## Data processing

Clean and transform the data.

In [9]:
# class DataCleaner(BaseEstimator, TransformerMixin):
#     def __init__(self):
#         return None
    
#     def fit(self, X, y=None):
#         return self

#     def transform(self, X, y=None):
#         df = X.copy()
#         df = self.add_dow_as_int(df)
#         df = self.add_missing_dow(df)
#         return df

#     def add_dow_as_int(self, X):
#         '''One hot encoding the day of the week'''
#         df = X.copy()
#         df['date'] = pd.to_datetime(df['date'])
#         one_hot = pd.get_dummies(df['dow'])
#         df = df.join(one_hot)
#         df = df.drop('dow',axis=1)
#         return df

#     def add_missing_dow(self, X):
#         ''' add encoding for missing dow in testing dataset'''
#         df = X.copy()
#         all_dow = [
#             'Monday',
#             'Tuesday',
#             'Wednesday',
#             'Thursday',
#             'Friday',
#             'Saturday',
#             'Sunday'
#         ]
#         dow_in_data = df.columns.drop(['date','hour','month']).to_list()
#         dow_not_in_data = np.setdiff1d(all_dow, dow_in_data, assume_unique=False)
#         for i in dow_not_in_data:
#             df[i] = False
#         df = df.drop(columns=["date"])
#         # Arrange columns and select final features
#         df = df[[
#             'lat',
#             'lon',
#             'hour',
#             'month',
#             'Monday',
#             'Tuesday',
#             'Wednesday',
#             'Thursday',
#             'Friday',
#             'Saturday',
#             'Sunday'
#         ]]
#         return df

## Train model

In [24]:
def get_dow(df: pd.DataFrame) -> pd.DataFrame:
    df['date'] = pd.to_datetime(df['date'])
    return df

def one_hot_encode(df: pd.DataFrame) -> pd.DataFrame:
    return pd.get_dummies(df, prefix=[""], prefix_sep="", columns = ['dow'])

def select_cols(df: pd.DataFrame) -> pd.DataFrame:
    return df[[
        'lat',
        'lon',
        'hour',
        'month',
        'Monday',
        'Tuesday',
        'Wednesday',
        'Thursday',
        'Friday',
        'Saturday',
        'Sunday'
    ]]

# build a random forest model
model = Pipeline(
    steps=[
        # ("get-dow", FunctionTransformer(
        #     lambda df: df.assign(
        #         date = lambda df: pd.to_datetime(df['date'])
        #     )
        # )),
        # ("one-hot-encode", FunctionTransformer(
        #     lambda df: pd.get_dummies(df, prefix=[""], prefix_sep="", columns = ['dow'])
        # )),
        # ("select-cols", FunctionTransformer(
        #     lambda df: df[[
        #         'lat',
        #         'lon',
        #         'hour',
        #         'month',
        #         'Monday',
        #         'Tuesday',
        #         'Wednesday',
        #         'Thursday',
        #         'Friday',
        #         'Saturday',
        #         'Sunday'
        #     ]]
        # )),
        # ("get-dow", FunctionTransformer(get_dow)),
        # ("one-hot-encode", FunctionTransformer(one_hot_encode)),
        # ("select-cols", FunctionTransformer(select_cols)),
        ("get-dow", FunctionTransformer(get_dow)),
        ("one-hot-encode", FunctionTransformer(one_hot_encode)),
        ("select-cols", FunctionTransformer(select_cols)),
        ("regressor", RandomForestRegressor(
            n_estimators=100, 
            random_state=0, 
            n_jobs=-1
        ))
    ]
)

model.fit(X_train, y_train)


Pipeline(steps=[('get-dow',
                 FunctionTransformer(func=<function get_dow at 0x7f2709026200>)),
                ('one-hot-encode',
                 FunctionTransformer(func=<function one_hot_encode at 0x7f2709025cf0>)),
                ('select-cols',
                 FunctionTransformer(func=<function select_cols at 0x7f2709025e10>)),
                ('regressor',
                 RandomForestRegressor(n_jobs=-1, random_state=0))])

In [25]:
# test the random forest model
y_pred = model.predict(X_test)

# compare predictions
test_mse = mean_squared_error(y_test, y_pred)
rmse = sqrt(test_mse)
print('RMSE: %f' % rmse)

RMSE: 2.825402


## Deploy model

Deploy the model with vetiver.

In [26]:
user_name="sam.edwardes"
pin_name = f"{user_name}/bikeshare-rf-python"

In [27]:
# convert the random forest model into a vetiver model
v = VetiverModel(
    model=model,
    model_name=pin_name,
    prototype_data=X_train.head(1),
    description="A model to predict the number of bikes that will be available."
)

In [28]:
# create a board on Posit Connect
board = pins.board_connect(
    server_url="https://colorado.posit.co/rsc",
    allow_pickle_read=True
)

In [29]:
# write the vetiver model as pin to Posit Connect
pin_read_write.vetiver_pin_write(
    board=board, 
    model=v
)

Model Cards provide a framework for transparent, responsible reporting. 
 Use the vetiver `.qmd` Quarto template as a place to start, 
 with vetiver.model_card()
Writing pin:
Name: 'sam.edwardes/bikeshare-rf-python'
Version: 20230609T210651Z-43166


In [30]:
board.pin_versions(pin_name)

,version
0,75565
1,75569
2,75571
3,75685
4,75693


In [31]:
?deploy_rsconnect

Signature:
deploy_rsconnect(
    connect_server: rsconnect.api.RSConnectServer,
    board,
    pin_name: str,
    version: str = None,
    extra_files: List[str] = None,
    new: bool = False,
    app_id: int = None,
    title: str = None,
    python: str = None,
    conda_mode: bool = False,
    force_generate: bool = False,
    log_callback: Callable = None,
    image: str = None,
)
Docstring:
Deploy to Posit Connect

Parameters
----------
connect_server : rsconnect.api.RSConnectServer
    Posit Connect Server
board :
    Pins board
pin_name : str
    Name of pin
version : str
    Version of pin
extra_files : typing.List[str]
    Any extra files to include
new : bool
    Force as a new deploy
app_id : int
    ID of an existing application to deploy new files for.
title : str
    Optional title for the deploy.
python : str
    Optional name of a Python executable
conda_mode : bool
    Use conda to build an environment.yml
force_generate : bool
    Force generating requirements.txt or 

In [32]:
# use Vetiver provided Posit Connect deployment function 
# to deploy the model as a FASTApi
deploy_rsconnect(
    connect_server=connect_server,
    board=board,
    pin_name=pin_name,
    version=board.pin_versions(pin_name).tail(1)["version"].values[0],
    title="Random Forest model for Bikeshare Python",
    app_id="28923e33-dcb6-4774-b753-bf1d4c367579"
    extra_files=["requirements.txt"]
)

             Consider creating a requirements.txt file instead.


Validating server... 	[OK]
Validating app mode... 	[OK]
Making bundle ... 	[OK]
Deploying bundle ... 	[OK]
Saving deployed information... 	[OK]
Building FastAPI application...
Bundle created with Python version 3.10.11 is compatible with environment Kubernetes::ghcr.io/rstudio/content-pro:r4.1.3-py3.10.11-ubuntu2204 with Python version 3.10.11 from /opt/python/3.10.11/bin/python3 
Bundle requested Python version 3.10.11; using /opt/python/3.10.11/bin/python3 from Kubernetes::ghcr.io/rstudio/content-pro:r4.1.3-py3.10.11-ubuntu2204 which has version 3.10.11
Determining session server location ...
2023/06/09 21:08:13.513702333 [rsc-session] Content GUID: 28923e33-dcb6-4774-b753-bf1d4c367579
2023/06/09 21:08:13.514145807 [rsc-session] Content ID: 16788
2023/06/09 21:08:13.514183089 [rsc-session] Bundle ID: 75694
2023/06/09 21:08:13.808438716 Running on host: python-environment-restore-9md4r-b57x6
Connecting to session server http://service-6d8ca117-a143-4bdd-bb69-05a4ee6ef272.rstudio-conne

## Tmp

In [ ]:
?deploy_rsconnect

In [ ]:
from pins import board_connect
board = board_connect(server_url='https://colorado.posit.co/rsc', allow_pickle_read=True)
o = board.pin_read("sam.edwardes/bikeshare-rf-python")

In [ ]:
import vetiver
z = vetiver.pin_read_write.vetiver_pin_read(board, pin_name)

In [ ]:
z.prototype()

In [ ]:
vetiver.write_fastapi.vetiver_write_app(board, pin_name)

In [ ]:
b = pins.board_connect(server_url='https://colorado.posit.co/rsc', allow_pickle_read=True)
v = VetiverModel.from_pin(b, 'sam.edwardes/bikeshare-rf-python', version = '75565')

In [ ]:
v.prototype()

In [ ]:
pprint(board.pin_meta(pin_name))

In [ ]:
board.pin_versions(pin_name)